# Creación del Scraper

## Importamos nuestras librerías 

In [68]:
import pandas as pd
import time 
from parsel import Selector
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

## Creamos la clase `googleMapsScraper` 

Contendrá las funciones principales para abrir el navegador, entrar a Google Maps, ejecutar una búsqueda, bajar el scroll hasta que dejen de salir resultados y devolver los nombres y URLs recopilados de los lugares.

In [69]:
class googleMapsScraper:
#################################################################
    def __init__(self): 
        #Inicializar el Driver
        driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()))
        self.driver = driver
#################################################################

    def searchPlaces(self, stringSearch):
        # Lista de resultados de la busqueda
        self.results = []

        try:
            searchText = self.driver.find_element(By.CLASS_NAME, "tactile-searchbox-input")
            searchText.send_keys(stringSearch)
            searchButton = self.driver.find_element(By.XPATH, '//html//body//div[3]//div[9]//div[3]//div[1]//div[1]//div[1]//div[2]//div[1]//button')
            searchButton.click()



            # Se tiene que generar antes de poder interactuar, por ello le damos 5 segundos
            time.sleep(5)
            # Ubicando elementos de la página 
            resultsBox= self.driver.find_element(By.XPATH, "/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]")
            # Acciones dentro del navegador
            action = ActionChains(self.driver)
            if (resultsBox.is_displayed() == True): # Si se están mostrando los resultados
                action.move_to_element(resultsBox) # Moverse a la sección de resultados
                initial_len = 0
                while(resultsBox.is_displayed() == True):
                    page_content = self.driver.page_source # Obteniendo contenido de la pagina inicial
                    response = Selector(page_content) 
                    initial_len = (len(response.xpath('//div[contains(@aria-label, "Resultados de")]/div/div[./a]'))) 
                    for i in range(5): # Para darle tiempo de actualizar busquedas y evitar que se traslapen
                        time.sleep(1) 
                        resultsBox.send_keys(Keys.PAGE_DOWN) # Scroll hacia abajo 
                    page_content = self.driver.page_source # Obteniendo contenido de la pagina después de actualizar
                    response = Selector(page_content) 
                    actual_len = (len(response.xpath('//div[contains(@aria-label, "Resultados de")]/div/div[./a]'))) # Obtenemos la cantidad de busquedas
                    if(initial_len == actual_len and actual_len!= 0):
                        break
                for el in response.xpath('//div[contains(@aria-label, "Resultados de")]/div/div[./a]'):
                    self.results.append({
                        'Nombre del lugar': el.xpath('./a/@aria-label').extract_first(''),
                        'Sitio en Google Maps': el.xpath('./a/@href').extract_first('')
                })
            return(self.results)
            
        except:
            pass
#################################################################
    def run(self, stringSearch):
        self.driver.get('https://www.google.com/maps')
        time.sleep(2)
        data = self.searchPlaces(stringSearch)
        return(data)

## Asignamos el input de la búsqueda

In [70]:
stringSearch = "  \" CDMX, GAM \"  "  +  "Krispy kreme"  

## Ejecutamos la clase anterior y guardamos los datos

In [74]:
mapsScraper = googleMapsScraper()
data = mapsScraper.run(stringSearch)

## Guardamos la información en un `DataFrame`

In [75]:
names = pd.DataFrame(data)
names

,Nombre del lugar,Sitio en Google Maps
0,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...
1,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...
2,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...
3,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...
4,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...
...,...,...
99,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...
100,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...
101,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...
102,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...


---

## Creamos la función `completeDataCollect()`

Está función es la más tardada y más importante, ya que se encarga de entrar a cada una de las URLs que obtuvimos anteriormente y recopilar toda la información importante del sitio en cuestión. Entre los datos recopilados tenemos la ubicación, cantidad de estrellas, conteo de reviews, sitio web, número de teléfono y horarios.

In [76]:
def completeDataCollect(url):

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=options)


    driver.get(url)


    page_content = driver.page_source # Obteniendo contenido de la 
    response = Selector(page_content)
    try:
        location = response.xpath('//button[contains(@aria-label, "Dirección:")]').xpath("@aria-label").extract_first().replace("Dirección:","")
    except:
        location = "N/A"

    try:  
        stars = response.xpath('//span[contains(@aria-label, "estrellas")]').xpath("@aria-label").extract_first().split()[0]
    except:
        stars = "N/A"

    try:
        reviewsCount = response.xpath('//button[contains(@aria-label, "opiniones")]').xpath("@aria-label").extract_first().split()[0]
    except:
        reviewsCount = "N/A"

    try:
        webSite = response.xpath('//a[contains(@aria-label, "Sitio web:")]').xpath("@href").extract_first()
    except:
        webSite = "N/A"

    try:
        phoneNumber = response.xpath('//button[contains(@aria-label, "Teléfono:")]').xpath("@aria-label").extract_first().replace("Teléfono: ","")
    except:
        phoneNumber = "N/A"

    try:
        sched = response.xpath('//div[contains(@aria-label, "horario de apertura durante la semana")]').xpath("@aria-label").extract_first("")
    except:
        sched = "N/A"
    
    return (location,stars,reviewsCount,webSite,phoneNumber,sched)

## Guardamos los datos de cada ubicación en una lista

In [77]:
results = []

for i in range(len(names)):
    results.append(completeDataCollect(names["Sitio en Google Maps"][i]))

## Generamos una lista para cada columna de información

In [79]:
addres_ = []
stars_ = []
reviewsCount_ = []
webSite_ = []
phone_ = []
sched_ = []

for i in range(len(names)):
    addres_.append(results[i][0])
    stars_.append(results[i][1])
    reviewsCount_.append(results[i][2])
    webSite_.append(results[i][3])
    phone_.append(results[i][4])
    sched_.append(results[i][5])

## Las asignamos a un diccionario con sus llaves correspondientes

In [80]:
dict_ = {"Dirección": addres_,
        "Calificación":stars_,
        "Total de Calificaciones": reviewsCount_,
        "Stitio web": webSite_,
        "Teléfono": phone_,
        "Horario": sched_
}

df_ = pd.DataFrame(dict_) 
df_

,Dirección,Calificación,Total de Calificaciones,Stitio web,Teléfono,Horario
0,"Colector 13 280, Revolución IMSS, Gustavo A. ...",4.2,820,http://www.krispykreme.mx/,55 5586 1150,"jueves, De 9:00 a 21:30; viernes, De 9:00 a 21..."
1,"Calz. San Juan de Aragón 516-Local S-2, DM Na...",4.2,788,http://www.krispykreme.mx/,55 5748 8066,"jueves, De 8:30 a 21:00; viernes, De 8:30 a 21..."
2,"Calle Riobamba 639, Magdalena de las Salinas,...",4.1,73,http://www.krispykreme.mx/,55 3350 7182,"jueves, De 8:30 a 21:30; viernes, De 8:30 a 21..."
3,"Calz de Guadalupe 431, Guadalupe Tepeyac, Gus...",4.2,367,http://www.krispykreme.mx/,55 5759 3450,"jueves, De 10:30 a 20:30; viernes, De 10:30 a ..."
4,"AV FORTUNA EJE 4 NORTE 334 LOCAL IA 08, Gusta...",5.0,Buscar,None,N/A,"jueves, De 9:00 a 20:30; viernes, De 9:00 a 20..."
...,...,...,...,...,...,...
99,"Vialidad de la Barranca 22, Hacienda de las P...",2.0,N/A,https://www.krispykreme.mx/2021/09/21/las-6-do...,N/A,"jueves, De 7:30 a 21:30; viernes, De 7:30 a 21..."
100,"C. Mexiquense 2, Col. Héroes de Tecamac, 5576...",3.5,12,None,N/A,"jueves, De 9:30 a 20:30; viernes, De 9:30 a 20..."
101,"Unnamed Road, Bosques de la Hacienda, 54768 C...",3.8,10,https://www.krispykreme.mx/,55 8797 8951,"jueves, De 9:00 a 21:00; viernes, De 9:00 a 21..."
102,"P.º de las Palomas 25, Las Alamedas, 52970 Cd...",4.0,32,None,N/A,"jueves, De 8:00 a 21:00; viernes, De 8:00 a 21..."


## Organizamos la información y terminamos el `DataFrame` final

In [81]:
data_df = pd.concat([names, df_], axis=1)
data_df

,Nombre del lugar,Sitio en Google Maps,Dirección,Calificación,Total de Calificaciones,Stitio web,Teléfono,Horario
0,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...,"Colector 13 280, Revolución IMSS, Gustavo A. ...",4.2,820,http://www.krispykreme.mx/,55 5586 1150,"jueves, De 9:00 a 21:30; viernes, De 9:00 a 21..."
1,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...,"Calz. San Juan de Aragón 516-Local S-2, DM Na...",4.2,788,http://www.krispykreme.mx/,55 5748 8066,"jueves, De 8:30 a 21:00; viernes, De 8:30 a 21..."
2,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...,"Calle Riobamba 639, Magdalena de las Salinas,...",4.1,73,http://www.krispykreme.mx/,55 3350 7182,"jueves, De 8:30 a 21:30; viernes, De 8:30 a 21..."
3,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...,"Calz de Guadalupe 431, Guadalupe Tepeyac, Gus...",4.2,367,http://www.krispykreme.mx/,55 5759 3450,"jueves, De 10:30 a 20:30; viernes, De 10:30 a ..."
4,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...,"AV FORTUNA EJE 4 NORTE 334 LOCAL IA 08, Gusta...",5.0,Buscar,None,N/A,"jueves, De 9:00 a 20:30; viernes, De 9:00 a 20..."
...,...,...,...,...,...,...,...,...
99,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...,"Vialidad de la Barranca 22, Hacienda de las P...",2.0,N/A,https://www.krispykreme.mx/2021/09/21/las-6-do...,N/A,"jueves, De 7:30 a 21:30; viernes, De 7:30 a 21..."
100,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...,"C. Mexiquense 2, Col. Héroes de Tecamac, 5576...",3.5,12,None,N/A,"jueves, De 9:30 a 20:30; viernes, De 9:30 a 20..."
101,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...,"Unnamed Road, Bosques de la Hacienda, 54768 C...",3.8,10,https://www.krispykreme.mx/,55 8797 8951,"jueves, De 9:00 a 21:00; viernes, De 9:00 a 21..."
102,Krispy Kreme,https://www.google.com/maps/place/Krispy+Kreme...,"P.º de las Palomas 25, Las Alamedas, 52970 Cd...",4.0,32,None,N/A,"jueves, De 8:00 a 21:00; viernes, De 8:00 a 21..."


---

In [82]:
data_df.to_csv(path_or_buf="/home/abdiel-fedora-linux/Documentos/DF_Collected.csv")

---